In [1]:
from src.rag_main.pre_retrieval import query_expansion, self_query

query = "What is RAG from Paul Iusztin?"
n = 3

queries = query_expansion(query, n)
metadata = self_query(query)

In [17]:
from src.rag_main.retrieval import top_k_matches

nk_matches = []
ids = []
k = 2
filters = {"author": metadata}

for query in queries:
    top_k = top_k_matches(query, k, filters=filters)
    for result in top_k:
        if result.id not in ids:
            nk_matches.append(result.payload["chunk"])
            ids.append(result.id)

In [42]:
from sentence_transformers.cross_encoder import CrossEncoder


model = CrossEncoder()

top_k = sorted(
    [(chunk, float(model.predict([(chunk, query)])[0])) for chunk in nk_matches],
    key=lambda x: x[1],
    reverse=True,
)

In [46]:
[match[0] for match in top_k]

["It cleans, chunks, and embeds every message and loads it to a vector DB in real-time.The training pipeline creates a custom instruction dataset based on your digital data. Fine-tune an LLM using Unsloth, AWS SageMaker, and Comet ML's experiment tracker. Evaluate the LLMs using Opik and save the best model to the Hugging Face model registry.The inference pipeline loads and quantizes the fine-tuned LLM from the model registry to the AWS SageMaker REST API. Enhance the prompts using RAG.",
 'It is an AI character that writes like yourself by incorporating your style, personality and voice into an LLM.Image by DALL-EWhy is this course different?By finishing the LLM Twin: Building Your Production-Ready AI Replica free course, you will learn how to design, train, and deploy a production-ready LLM twin of yourself powered by LLMs, vector DBs, and LLMOps good practices.Why should you care? No more isolated scripts or Notebooks!']

In [35]:
float(top_k[0][1][0])

-6.721551418304443

In [2]:
import textwrap

expand_to_n = 3
separator = "#"
question = "What is RAG?"

prompt = textwrap.dedent(
    f"""\
    You are an AI language model assistant. Your task is to generate {expand_to_n} different
    versions of the given user question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question, your goal is to help the user
    overcome some of the limitations of the distance-based similarity search.

    Provide these alternative questions separated by '{separator}' and do not use nothing at
    the start. For example, '1: ...' or 'Version 1: ...' is not correct. Just write the
    alternative questions separated.

    Original question: {question}.
"""
)

In [3]:
prompt

"You are an AI language model assistant. Your task is to generate 3 different\nversions of the given user question to retrieve relevant documents from a vector database.\nBy generating multiple perspectives on the user question, your goal is to help the user\novercome some of the limitations of the distance-based similarity search.\n\nProvide these alternative questions separated by '#' and do not use nothing at\nthe start. For example, '1: ...' or 'Version 1: ...' is not correct. Just write the\nalternative questions separated.\n\nOriginal question: What is RAG?.\n"